In [1]:
pip install spotipy

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.5/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import spotipy
import spotipy.util as util
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from spotipy.oauth2 import SpotifyClientCredentials

In [42]:
cid = "24b738724d62406d8f246455d65ad51f"
secret = "0f5621b1f4cd4305b937b7d0b53575b2"
username = "Spotify"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
playlists = sp.user_playlists(username)
playlist_names = ['Coffee Table Jazz', 'Rap Caviar', 'Classical Essentials', 'Rock Classics']
file_names = ['./coffee_table_jazz.csv', './rap_caviar.csv', './classical_essentials.csv', './rock_classics.csv']
playlist_ids = ['37i9dQZF1DWVqfgj8NZEp1', '37i9dQZF1DX0XUsuxWHRQd', '37i9dQZF1DWWEJlAGA9gs0', '37i9dQZF1DWXRqgorJj26U']

In [58]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "track_name", "track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[3:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    playlist_df.drop(columns=['track_id'], inplace=True)
        
    return playlist_df

In [ ]:
for i, playlist_id in enumerate(playlist_ids):
    track_list = analyze_playlist(username, playlist_id)
    print(track_list)
    track_list.to_csv(file_names[i]) 